Soal 1

In [56]:
import pandas as pd

class MarketingDataETL:
    def __init__(self, file_path):
        self.file_path = file_path
        self.raw_data = None
        self.transformed_data = None

    def extract(self):
        try:
            # Read data from CSV file
            self.raw_data = pd.read_csv(self.file_path, sep=';')
            print("Data berhasil diekstrak dari", self.file_path)
        except FileNotFoundError:
            print("File tidak ditemukan. Harap berikan path file yang valid.")
        except Exception as e:
            print("Terjadi kesalahan saat ekstraksi data:", e)

    def transform(self):
        if self.raw_data is None:
            print("Tidak ada data untuk diubah. Silakan jalankan extract() terlebih dahulu.")
            return

        try:
            # Mengubah format tanggal pada kolom 'purchase_date' menjadi 'YYYY-MM-DD'
            self.raw_data['purchase_date'] = pd.to_datetime(self.raw_data['purchase_date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
            self.transformed_data = self.raw_data  # Menyimpan data yang telah ditransformasi
            print("Data berhasil diubah.")
        except Exception as e:
            print("Terjadi kesalahan saat transformasi data:", e)

    def sort_by_customer_id(self):
        if self.transformed_data is None:
            print("Tidak ada data untuk diurutkan. Silakan jalankan transform() terlebih dahulu.")
            return

        try:
            # Urutkan DataFrame berdasarkan kolom 'customer_id'
            self.transformed_data = self.transformed_data.sort_values(by='customer_id')
            print("Data berhasil diurutkan berdasarkan customer_id.")
        except Exception as e:
            print("Terjadi kesalahan saat mengurutkan data:", e)

    def remove_empty_rows(self):
        if self.transformed_data is None:
            print("Tidak ada data untuk dihapus baris kosong. Silakan jalankan transform() terlebih dahulu.")
            return

        try:
            # Hapus baris yang memiliki nilai kosong (NaN)
            self.transformed_data = self.transformed_data.dropna()
            print("Baris kosong berhasil dihapus.")
        except Exception as e:
            print("Terjadi kesalahan saat menghapus baris kosong:", e)

    def store(self, output_file):
        if self.transformed_data is None:
            print("Tidak ada data yang disimpan. Silakan jalankan transform() terlebih dahulu.")
            return

        try:
            # Menyimpan data yang telah ditransformasi ke dalam file CSV
            self.transformed_data.to_csv(output_file, index=False)
            print("Data berhasil disimpan ke", output_file)
        except Exception as e:
            print("Terjadi kesalahan saat menyimpan data:", e)

# Contoh penggunaan:
etl_processor = MarketingDataETL('https://raw.githubusercontent.com/Anissls333/TPG/main/marketing_data.csv')
etl_processor.extract()
etl_processor.transform()
etl_processor.sort_by_customer_id()
etl_processor.remove_empty_rows()
etl_processor.store("transformed_marketing_data.csv")

Data berhasil diekstrak dari https://raw.githubusercontent.com/Anissls333/TPG/main/marketing_data.csv
Data berhasil diubah.
Data berhasil diurutkan berdasarkan customer_id.
Baris kosong berhasil dihapus.
Data berhasil disimpan ke transformed_marketing_data.csv


In [57]:
df = pd.read_csv('transformed_marketing_data.csv')
print(df)

  customer_id purchase_date product_category  amount_spent
0        C001    2023-01-03      Electronics        250.00
1        C001    2023-04-03            Books         20.00
2        C002    2023-02-03            Books         45.50
3        C002    2023-05-03      Electronics        525.00
4        C003    2023-03-03    Home & Garden        150.75
5        C005    2023-07-03           Beauty         75.00


Soal 2

In [58]:
class TargetedMarketingETL(MarketingDataETL):
    def __init__(self, file_path):
        super().__init__(file_path)
        self.segmented_data = None

    def segment_customers(self, criteria):
        if self.transformed_data is None:
            print("Tidak ada data untuk disegmentasi. Silakan jalankan transform() terlebih dahulu.")
            return

        try:
            # Implementasi logika segmentasi pelanggan berdasarkan kriteria tertentu
            if criteria == 'total_spending':
                self.segmented_data = self.transformed_data.groupby('customer_id')['amount_spent'].sum().reset_index()
                self.segmented_data.columns = ['customer_id', 'total_spending']
            elif criteria == 'product_category':
                # Contoh: Mengelompokkan pelanggan berdasarkan kategori produk yang dibeli
                self.segmented_data = self.transformed_data.groupby(['customer_id', 'product_category']).size().unstack(fill_value=0)
            else:
                print("Kriteria tidak valid untuk segmentasi pelanggan.")
                return

            print("Pelanggan berhasil disegmentasi berdasarkan", criteria)
            print(self.segmented_data)  # Cetak hasil segmentasi
        except Exception as e:
            print("Terjadi kesalahan saat segmentasi pelanggan:", e)

    # Override metode transform untuk menambahkan logika segmentasi pelanggan
    def transform(self):
        super().transform()  # Memanggil metode transform dari parent class

        if self.segmented_data is not None:
            # Lakukan operasi transformasi tambahan jika diperlukan
            print("Melakukan transformasi tambahan untuk segmentasi pelanggan.")
            print(self.segmented_data)  # Cetak hasil segmentasi tambahan


# Contoh penggunaan:
targeted_etl_processor = TargetedMarketingETL('https://raw.githubusercontent.com/Anissls333/TPG/main/marketing_data.csv')
targeted_etl_processor.extract()
targeted_etl_processor.transform()
targeted_etl_processor.sort_by_customer_id()
targeted_etl_processor.remove_empty_rows()
targeted_etl_processor.segment_customers('total_spending')
targeted_etl_processor.store("segmented_marketing_data.csv")


Data berhasil diekstrak dari https://raw.githubusercontent.com/Anissls333/TPG/main/marketing_data.csv
Data berhasil diubah.
Data berhasil diurutkan berdasarkan customer_id.
Baris kosong berhasil dihapus.
Pelanggan berhasil disegmentasi berdasarkan total_spending
  customer_id  total_spending
0        C001          270.00
1        C002          570.50
2        C003          150.75
3        C005           75.00
Data berhasil disimpan ke segmented_marketing_data.csv
